In [ ]:
# import necessary libraries
import pandas as pd
import pandas as pd
import numpy as np
import ast
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
playerMatchDetailsPerformanceDF = pd.read_csv('../aggregateMatchAndTimelineLarge.csv')
summonerRanksDF = pd.read_csv('../summonerRanksLarge.csv')
summonerRanksDF = summonerRanksDF[['summonerId', 'queueType', 'tier', 'rank', 'leaguePoints']] # just selecting necessary sets of data

print(playerMatchDetailsPerformanceDF.shape)
print(summonerRanksDF.shape)

for aCol in playerMatchDetailsPerformanceDF.columns:
    print(aCol)

print("-------------------------------")
for rCol in summonerRanksDF.columns:
    print(rCol)

In [ ]:
# join summonerRanks with aggregateMatchDF
playerMatchDetailsPerformanceWithRanksDF = playerMatchDetailsPerformanceDF.merge(summonerRanksDF, on='summonerId', how='inner')
print(playerMatchDetailsPerformanceWithRanksDF.shape)

# count of total players in each game; both teams; can be used for filtering matches or check sanity tests
matchPlayerCounts = playerMatchDetailsPerformanceWithRanksDF.groupby(['gameId'])['summonerId'].count().reset_index()
matchPlayerCounts.rename(columns={'summonerId': 'matchPlayerCounts'}, inplace=True)

# adds the matchPlayerCounts to the dataframe in use
playerMatchDetailsPerformanceWithRanksDF = playerMatchDetailsPerformanceWithRanksDF.merge(matchPlayerCounts, on='gameId', how='inner')
playerMatchDetailsPerformanceWithRanksDF.head()

# apply the filter of 10 players in a team to prevent cases of a single player being AFK
valid_player_match_details_df = playerMatchDetailsPerformanceWithRanksDF[playerMatchDetailsPerformanceWithRanksDF['matchPlayerCounts'] == 10]
print("Number of matches after player count filter: ", valid_player_match_details_df.shape[0] / 10) # divide by 10 because there are 10 rows for each match; corresponding to players



In [ ]:
# of the 7220 player records (i.e. 722 matches for 10 players in each team), the RECORDS for RANKED games should total to this number of 7220
valid_player_match_details_df['queueType'].value_counts()

## here 5838 + 1385 added to 7220. don't be confused that RANKED SOLO and RANKED FLEX always has to be multiple of 10. 
# it is possible that RANKED SOLO is being matched with opponent of RANKED FLEX 5


In [ ]:
valid_player_match_details_df['summonerId'].value_counts()

In [ ]:
# convert player ranks to numerical ranks
# join the tier and rank to yield a combined value
valid_player_match_details_df['tierRank'] = valid_player_match_details_df['tier'] + '-' + valid_player_match_details_df['rank']

rankedDict = {
    'NA':0,
    'IRON-IV':1,
    'IRON-III':2,
    'IRON-II':3,
    'IRON-I':4,
    'BRONZE-IV':5,
    'BRONZE-III':6,
    'BRONZE-II':7,
    'BRONZE-I':8,
    'SILVER-IV':9,
    'SILVER-III':10,
    'SILVER-II':11,
    'SILVER-I':12,
    'GOLD-IV':13,
    'GOLD-III':14,
    'GOLD-II':15,
    'GOLD-I':16,
    'PLATINUM-IV':17,
    'PLATINUM-III':18,
    'PLATINUM-II':19,
    'PLATINUM-I':20,
    'EMERALD-IV':21,
    'EMERALD-III':22,
    'EMERALD-II':23,
    'EMERALD-I':24,
    'DIAMOND-IV':25,
    'DIAMOND-III':26,
    'DIAMOND-II':27,
    'DIAMOND-I':28,
    'MASTER-I':29,
    'GRANDMASTER-I':30,
    'CHALLENGER-I':31
}

# for calculating a numerical rank value for a player of a given match
def calculateNumericalRank(row):
    return rankedDict[row['tierRank']]

valid_player_match_details_df['playerNumericalRank'] = valid_player_match_details_df.apply(calculateNumericalRank, axis=1)

# averaging the rank of all 10 players in a match
matchAverageRankDF = valid_player_match_details_df.groupby(['gameId'])['playerNumericalRank'].mean().reset_index()
matchAverageRankDF.rename(columns={'playerNumericalRank': 'matchAverageRank'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(matchAverageRankDF, on='gameId', how='inner')

# averaging the rank of a given team in a match; 5 players in the team
teamAverageRankDF = valid_player_match_details_df.groupby(['gameId', 'teamId'])['playerNumericalRank'].mean().reset_index()
teamAverageRankDF.rename(columns={'playerNumericalRank': 'teamAverageRank'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(teamAverageRankDF, on=['gameId', 'teamId'], how='inner')

# summing up total goldEarned by a team in a match
totalTeamGold = valid_player_match_details_df.groupby(['gameId', 'teamId'])['goldEarned'].sum().reset_index()
totalTeamGold.rename(columns={'goldEarned': 'totalTeamGold'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamGold, on=['gameId', 'teamId'], how='inner')

# another column for total gold per minute for the team
valid_player_match_details_df['totalTeamGPM'] = np.ceil(valid_player_match_details_df['totalTeamGold'] / valid_player_match_details_df['gameDuration'])

# summing up the total kills by a team in a match
totalTeamKills = valid_player_match_details_df.groupby(['gameId', 'teamId'])['kills'].sum().reset_index()
totalTeamKills.rename(columns={'kills': 'totalTeamKills'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamKills, on=['gameId', 'teamId'], how='inner')

# another column for total kills per minute for the team
valid_player_match_details_df['totalTeamKPM'] = valid_player_match_details_df['totalTeamKills'] / valid_player_match_details_df['gameDuration']

# summing up the total deaths of a team in a match
totalTeamDeaths = valid_player_match_details_df.groupby(['gameId', 'teamId'])['deaths'].sum().reset_index()
totalTeamDeaths.rename(columns={'deaths': 'totalTeamDeaths'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamDeaths, on=['gameId', 'teamId'], how='inner')

# summing up the total assists by a team in a match
totalTeamAssists = valid_player_match_details_df.groupby(['gameId', 'teamId'])['assists'].sum().reset_index()
totalTeamAssists.rename(columns={'assists': 'totalTeamAssists'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamAssists, on=['gameId', 'teamId'], how='inner')

valid_player_match_details_df['totalTeamAPM'] = valid_player_match_details_df['totalTeamAssists'] / valid_player_match_details_df['gameDuration']

# summing up the total epicMonsterKills by a team in a match
totalTeamEpicMonsterKills = valid_player_match_details_df.groupby(['gameId', 'teamId'])['epicMonsterKills'].sum().reset_index()
totalTeamEpicMonsterKills.rename(columns={'epicMonsterKills': 'totalTeamEpicMonsterKills'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamEpicMonsterKills, on=['gameId', 'teamId'], how='inner')

# summing up the total towers taken by a team in a match
totalTeamTowerKills = valid_player_match_details_df.groupby(['gameId', 'teamId'])['turretKills'].sum().reset_index()
totalTeamTowerKills.rename(columns={'turretKills': 'totalTeamTurretKills'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamTowerKills, on=['gameId', 'teamId'], how='inner')

# summing up vision score for a team in a match
totalTeamVisionScore = valid_player_match_details_df.groupby(['gameId', 'teamId'])['visionScore'].sum().reset_index()
totalTeamVisionScore.rename(columns={'visionScore': 'totalTeamVisionScore'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamVisionScore, on=['gameId', 'teamId'], how='inner')

# another metric for vision score of the team per minute
valid_player_match_details_df['totalTeamVPM'] = valid_player_match_details_df['totalTeamVisionScore'] / valid_player_match_details_df['gameDuration']

# team average vision score in a match
averageTeamVisionScore = valid_player_match_details_df.groupby(['gameId', 'teamId'])['visionScore'].mean().reset_index()
averageTeamVisionScore.rename(columns={'visionScore': 'averageTeamVisionScore'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(averageTeamVisionScore, on=['gameId', 'teamId'], how='inner')

# team total champ experience in a match
totalTeamChampExperience = valid_player_match_details_df.groupby(['gameId', 'teamId'])['champExperience'].sum().reset_index()
totalTeamChampExperience.rename(columns={'champExperience': 'totalTeamChampExperience'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamChampExperience, on=['gameId', 'teamId'], how='inner')

# another column for total champ experience per minute per team
valid_player_match_details_df['totalTeamExpPerMin'] = valid_player_match_details_df['totalTeamChampExperience'] / valid_player_match_details_df['gameDuration']

# average champ experience per team
averageTeamChampExperience = valid_player_match_details_df.groupby(['gameId', 'teamId'])['champExperience'].mean().reset_index()
averageTeamChampExperience.rename(columns={'champExperience': 'averageTeamChampExperience'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(averageTeamChampExperience, on=['gameId', 'teamId'], how='inner')

# total minions killed by a team in a match
totalTeamMinionsKilled = valid_player_match_details_df.groupby(['gameId', 'teamId'])['totalMinionsKilled'].sum().reset_index()
totalTeamMinionsKilled.rename(columns={'totalMinionsKilled': 'totalTeamMinionsKilled'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamMinionsKilled, on=['gameId', 'teamId'], how='inner')

valid_player_match_details_df['totalTeamMinionsPerMin'] = valid_player_match_details_df['totalTeamMinionsKilled'] / valid_player_match_details_df['gameDuration']

# average minions killed per team
averageTeamMinionsKilled = valid_player_match_details_df.groupby(['gameId', 'teamId'])['totalMinionsKilled'].mean().reset_index()
averageTeamMinionsKilled.rename(columns={'totalMinionsKilled': 'averageTeamMinionsKilled'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(averageTeamMinionsKilled, on=['gameId', 'teamId'], how='inner')

print(valid_player_match_details_df.shape)

valid_player_match_details_df[['gameId', 'teamId', 'win', 'participantId', 'teamPosition', 'kills', 'deaths', 'epicMonsterKills','assists', 'turretKills', 'playerNumericalRank', 'teamAverageRank', 'matchAverageRank', 'totalTeamKills', 'totalTeamDeaths', 'totalTeamAssists', 'totalTeamEpicMonsterKills', 'totalTeamTurretKills']].head(20)

In [ ]:
for col in valid_player_match_details_df.columns:
    print(col)

In [ ]:
# create a sub-set of the dataframe from the valid matches that could be used to map assists
assistsMapDF = valid_player_match_details_df[[
    'gameId', 'teamId', 'win', 'teamPosition', 'teamAverageRank', 'matchAverageRank', # overall match info 
    'participantId', 'kills', 'deaths', 'assists', 'participantsAssisted', 'towerKillsAssisted', 'monsterKillsAssisted', 'participantsAssistedWithPressure', # participant performance
    'totalTeamKills', 'totalTeamAssists', 'totalTeamTurretKills', 'totalTeamEpicMonsterKills', # team performance
    'totalTeamGPM', 'totalTeamKPM', 'totalTeamExpPerMin', 'totalTeamVPM', 'totalTeamAPM', 'totalTeamMinionsPerMin' # per min team performance
]]

# convert the string representation of assists list into actual lists using ast
assistsMapDF['participantsAssisted'] = assistsMapDF['participantsAssisted'].apply(ast.literal_eval)
assistsMapDF['towerKillsAssisted'] = assistsMapDF['towerKillsAssisted'].apply(ast.literal_eval)
assistsMapDF['monsterKillsAssisted'] = assistsMapDF['monsterKillsAssisted'].apply(ast.literal_eval)
assistsMapDF['participantsAssistedWithPressure'] = assistsMapDF['participantsAssistedWithPressure'].apply(ast.literal_eval)

# concatenate the 4 lists into a single list
assistsMapDF['totalAssisted'] = assistsMapDF.apply(lambda row: [item for sublist in (row['participantsAssisted'], row['towerKillsAssisted'], row['monsterKillsAssisted'], row['participantsAssistedWithPressure']) for item in sublist], axis=1)

# new column to store the count of pressure assists per team
totalTeamPressureAssists = assistsMapDF.groupby(['teamId', 'gameId'])['participantsAssistedWithPressure'].apply(lambda x: x.apply(len).sum()).reset_index()
totalTeamPressureAssists.rename(columns={'participantsAssistedWithPressure': 'totalTeamPressureAssists'}, inplace=True)
assistsMapDF = assistsMapDF.merge(totalTeamPressureAssists, on=['gameId', 'teamId'], how='inner')

# new column that sums up team assists of all kinds; used for normalization
assistsMapDF['totalTeamAllAssists'] = assistsMapDF['totalTeamAssists'] + assistsMapDF['totalTeamEpicMonsterKills'] + assistsMapDF['totalTeamTurretKills'] + assistsMapDF['totalTeamPressureAssists']
assistsMapDF['totalTeamAllKills'] = assistsMapDF['totalTeamKills'] + assistsMapDF['totalTeamTurretKills'] + assistsMapDF['totalTeamEpicMonsterKills']
assistsMapDF['intensity'] = assistsMapDF['totalTeamAllAssists'] / assistsMapDF['totalTeamAllKills']

assistsMapDF.head(20)

In [ ]:
## based on the totalAssisted column, create another column that holds a dictionary such that (key,value) such that 
# key = participantId that was assisted and 
# value = number of times that participant was assisted

def convertListToDict(list):
    myDict = {}
    for item in list:
        if item not in myDict.keys():
            myDict[item] = 1
        else:
            myDict[item] += 1
    return myDict

assistsMapDF['totalAssistsMap'] = assistsMapDF['totalAssisted'].apply(convertListToDict)
assistsMapDF


In [ ]:
# computing total indegree and total outdegree of individual players in the team
from collections import defaultdict

# default empty dictionary to use later for indegree assignment
indegree = defaultdict(int)

# Calculate playerOutdegree and update indegree counts; alternatively you can use it to using len of totalAssisted
assistsMapDF['playerTotalOutdegree'] = assistsMapDF['totalAssistsMap'].apply(lambda x: sum(x.values()))

for _, row in assistsMapDF.iterrows():
    for teammate, assist_count in row['totalAssistsMap'].items():
        indegree[(row['gameId'], teammate)] += assist_count

# Add playerIndegree to the DataFrame
assistsMapDF['playerTotalIndegree'] = assistsMapDF.apply(lambda row: indegree[(row['gameId'], row['participantId'])], axis=1)

In [ ]:
## computing in-degree centrality of a team
# step 1; compute max of team in-degree
teamMaxIndegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalIndegree'].max().reset_index()
teamMaxIndegree.rename(columns={'playerTotalIndegree': 'teamMaxIndegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamMaxIndegree, on=['gameId', 'teamId'], how='inner')

# step 2; compute sum of all team members in-degree
teamTotalIndegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalIndegree'].sum().reset_index()
teamTotalIndegree.rename(columns={'playerTotalIndegree': 'teamTotalIndegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamTotalIndegree, on=['gameId', 'teamId'], how='inner')

# step 3; sum up the difference between max and individual in-degree and sum it up to another column
assistsMapDF['playerDiffMaxTeamIndegree'] = assistsMapDF['teamMaxIndegree'] - assistsMapDF['playerTotalIndegree']
playerDiffMaxTeamIndegreeSum = assistsMapDF.groupby(['gameId', 'teamId'])['playerDiffMaxTeamIndegree'].sum().reset_index()
playerDiffMaxTeamIndegreeSum.rename(columns={'playerDiffMaxTeamIndegree': 'playerDiffMaxTeamIndegreeSum'}, inplace=True)
assistsMapDF = assistsMapDF.merge(playerDiffMaxTeamIndegreeSum, on=['gameId', 'teamId'], how='inner')

# step 4; the summed up column to be divided by (4 * sum of all indegrees)
assistsMapDF['teamIndegreeCentrality'] = assistsMapDF['playerDiffMaxTeamIndegreeSum'] / ( 4 * assistsMapDF['teamTotalIndegree'] )

In [ ]:
## computing out-degree centrality of a team and weight centralization of the team
# step 1; compute max of team's out-degree
teamMaxOutdegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalOutdegree'].max().reset_index()
teamMaxOutdegree.rename(columns={'playerTotalOutdegree': 'teamMaxOutdegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamMaxOutdegree, on=['gameId', 'teamId'], how='inner')

# step 2; compute sum of all team members out-degree
teamTotalOutdegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalOutdegree'].sum().reset_index()
teamTotalOutdegree.rename(columns={'playerTotalOutdegree': 'teamTotalOutdegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamTotalOutdegree, on=['gameId', 'teamId'], how='inner')

# step 3; sum up the difference between max and individual out-degree and sum it up to another column
assistsMapDF['playerDiffMaxTeamOutdegree'] = assistsMapDF['teamMaxOutdegree'] - assistsMapDF['playerTotalOutdegree']
playerDiffMaxTeamIndegreeSum = assistsMapDF.groupby(['gameId', 'teamId'])['playerDiffMaxTeamOutdegree'].sum().reset_index()
playerDiffMaxTeamIndegreeSum.rename(columns={'playerDiffMaxTeamOutdegree': 'playerDiffMaxTeamOutdegreeSum'}, inplace=True)
assistsMapDF = assistsMapDF.merge(playerDiffMaxTeamIndegreeSum, on=['gameId', 'teamId'], how='inner')

# step 4; the summed up column to be divided by (4 * sum of all indegrees)
assistsMapDF['teamOutdegreeCentrality'] = assistsMapDF['playerDiffMaxTeamOutdegreeSum'] / ( 4 * assistsMapDF['teamTotalIndegree'] ) # 4 = N-1; totalTeamIndegree is the same as totalTeamOutdegree; so you can use either of them

# for weight centralization
assistsMapDF['teamWeightCentralization'] = assistsMapDF['playerDiffMaxTeamOutdegreeSum'] / (19 * assistsMapDF['teamTotalIndegree'])

assistsMapDF

In [ ]:
assistsMapDF[assistsMapDF['gameId'] == 'EUW1_6895619260']

In [ ]:
# just create a dataframe with participant id and their positions
teamPositionsDF = assistsMapDF[['gameId', 'teamId', 'participantId', 'teamPosition']]
teamPositionsDF.drop_duplicates(inplace=True)
print(teamPositionsDF.shape)

In [ ]:
assistsMapDF.columns

In [ ]:
# expanding the totalAssistsMap columns to reflect the weights between nodes of players
edgesDF_rows = []

# Iterate through each row in the DataFrame
for _, row in assistsMapDF.iterrows():
    gameId = row['gameId']
    teamId = row['teamId']
    teamAverageRank = row['teamAverageRank']
    matchAverageRank = row['matchAverageRank']
    participant = row['participantId']
    totalAssistsMap = row['totalAssistsMap']
    teamIndegreeCentrality = row['teamIndegreeCentrality']
    teamOutdegreeCentrality = row['teamOutdegreeCentrality']
    teamTotalAllAssists = row['totalTeamAllAssists']
    win = row['win']
    teamGPM = row['totalTeamGPM']
    teamAverageRank = row['teamAverageRank']
    matchAverageRank = row['matchAverageRank']
    
    # Iterate through each key-value pair in the totalAssistsMap
    for to_participant, weight in totalAssistsMap.items():
        edgesDF_rows.append({
            'gameId': gameId,
            'teamId': teamId,
            'win': win,
            'fromParticipantId': participant,
            'toParticipantId': to_participant,
            'weight': weight,
            'teamIndegreeCentrality': teamIndegreeCentrality,
            'teamOutdegreeCentrality': teamOutdegreeCentrality,
            'teamTotalAllAssists': teamTotalAllAssists,
            'teamGPM': teamGPM,
            'teamAverageRank': teamAverageRank,
            'matchAverageRank': matchAverageRank
        })

# Create a new DataFrame from the expanded rows
edgesDF = pd.DataFrame(edgesDF_rows)

edgesDF['uniqueTeamId'] = edgesDF['gameId'].astype(str) + '_' + edgesDF['teamId'].astype(str)
edgesDF['normalizedWeight'] = edgesDF['weight'] / edgesDF['teamTotalAllAssists']

edgesDF.head(20)

In [ ]:
# merge team position of respective from participant id to participant id
edgesDF = edgesDF.merge(teamPositionsDF, left_on=['gameId', 'teamId', 'fromParticipantId'], right_on=['gameId', 'teamId', 'participantId'], how='inner')
edgesDF.rename(columns={'teamPosition': 'fromPlayerPosition'}, inplace=True)

edgesDF = edgesDF.merge(teamPositionsDF, left_on=['gameId', 'teamId', 'toParticipantId'], right_on=['gameId', 'teamId', 'participantId'], how='inner')
edgesDF.rename(columns={'teamPosition': 'toPlayerPosition'}, inplace=True)

In [ ]:
# computing resistance for the team by drawing a network for each team in each match and calculate resistance
def CalcResistance(eigenvalues,N=5):
    eigval = eigenvalues.tolist()
    roundedEigval = [round(val, 2) for val in eigval]
    roundedEigval = [i for i in roundedEigval if i != 0]

    summ = 0
    for e in roundedEigval:
        summ = summ + (1/e)
    R = 5 * summ
    return R

# for calculating graph resistance of teams using the edgesDF
uniqueTeamIdList = edgesDF['uniqueTeamId'].tolist()
uniqueTeamIdList = list(set(uniqueTeamIdList))
resistanceDict = {}

for idx, uid in enumerate(uniqueTeamIdList):
    uniqueTeamDF = edgesDF[edgesDF['uniqueTeamId'] == uid]
    
    # create a network graph object using network library; compute a
    G = nx.from_pandas_edgelist(uniqueTeamDF,'fromParticipantId','toParticipantId', edge_attr='weight', create_using=nx.MultiGraph(directed=True))
    A = nx.adjacency_matrix(G) # adjacency matrix
    D = np.diag(np.sum(np.array(A.todense()), axis=1)) # degree matrix
    L = D - A # laplacian matrix
    e, v = np.linalg.eig(L) # eigen values

    resistanceValue = CalcResistance(e)
    resistanceDict[uid] = round(resistanceValue,4)

teamResistance = pd.DataFrame(resistanceDict.items(), columns=['uniqueTeamId', 'resistance'])

# merge teamResistance DF into edgesDF using uniqueTeamId key
edgesDF = edgesDF.merge(teamResistance, on='uniqueTeamId', how='inner')
print(edgesDF.shape)

# merge teamResistance to assistsMapDF
assistsMapDF['uniqueTeamId'] = assistsMapDF['gameId'].astype(str) + '_' + assistsMapDF['teamId'].astype(str)
assistsMapDF = assistsMapDF.merge(teamResistance, on='uniqueTeamId', how='inner')
assistsMapDF.head(10)


In [ ]:
def convertNumericalRank(nRank):
    if nRank > 30:
        return 'CHALLENGER'
    elif nRank <=30 and nRank > 29:
        return 'GRANDMASTER'
    elif nRank <= 29 and nRank > 28:
        return 'MASTER'
    elif nRank <= 28 and nRank > 24:
        return 'DIAMOND'
    elif nRank <= 24 and nRank > 20:
        return 'EMERALD'
    elif nRank <= 20 and nRank > 16:
        return 'PLATINUM'
    elif nRank <= 16 and nRank > 12:
        return 'GOLD'
    elif nRank <= 12 and nRank > 8:
        return 'SILVER'
    elif nRank <= 8 and nRank > 4:
        return 'BRONZE'
    else:
        return 'IRON'
    
edgesDF['gameRank'] = edgesDF['matchAverageRank'].apply(convertNumericalRank)
edgesDF['teamRank'] = edgesDF['teamAverageRank'].apply(convertNumericalRank)

print(edgesDF.columns)

In [ ]:
gameIds = edgesDF['gameId'].tolist()
gameIds = list(set(gameIds))
gameIds = sorted(gameIds)

# use incremental ranges of gameIds to save figure locally
for idx, gameId in enumerate(gameIds[3200:3490]):
    # team level data of two teams in the same match-up
    blueTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 100)]
    blueTeamDF = blueTeamDF[(blueTeamDF['toParticipantId'] >= 1) & (blueTeamDF['toParticipantId'] <= 5)] # additional filter to remove any steals or lease

    redTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 200)]
    redTeamDF = redTeamDF[(redTeamDF['toParticipantId'] >= 6) & (redTeamDF['toParticipantId'] <= 10)] # additional filter to remove any steal or lease

    blueTeamEdges = [ min(100 * i, 5) for i in blueTeamDF['normalizedWeight'] ]
    redTeamEdges = [ min(100 * i, 5) for i in redTeamDF['normalizedWeight'] ]

    matchRank = blueTeamDF['gameRank'].iloc[0]
    titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'

    blueTeamAvgRank = blueTeamDF['teamRank'].iloc[0]
    blueTeamWins = blueTeamDF['win'].iloc[0]
    blueTeamGPM = blueTeamDF['teamGPM'].iloc[0]
    blueTeamInD = blueTeamDF['teamIndegreeCentrality'].iloc[0]
    blueTeamOutD = blueTeamDF['teamOutdegreeCentrality'].iloc[0]
    blueTeamRes = blueTeamDF['resistance'].iloc[0]
    
    blueTeamText = f'WIN: {blueTeamWins} | Team Rank: {blueTeamAvgRank} | Gold Per Min.: {blueTeamGPM} \n In-Degree: {blueTeamInD:.3f} | Out-Degree: {blueTeamOutD:.3f} | Resistance: {blueTeamRes:.3f}'
    
    redTeamAvgRank = redTeamDF['teamRank'].iloc[0]
    redTeamWins = redTeamDF['win'].iloc[0]
    redTeamGPM = redTeamDF['teamGPM'].iloc[0]
    redTeamGPM = redTeamDF['teamGPM'].iloc[0]
    redTeamInD = redTeamDF['teamIndegreeCentrality'].iloc[0]
    redTeamOutD = redTeamDF['teamOutdegreeCentrality'].iloc[0]
    redTeamRes = redTeamDF['resistance'].iloc[0]

    redTeamText = f'WIN: {redTeamWins} | Team Rank: {redTeamAvgRank} | Gold Per Min.: {redTeamGPM} \n In-Degree: {redTeamInD:.3f} | Out-Degree: {redTeamOutD:.3f} | Resistance: {redTeamRes:.3f}'

    # create network graph objects
    BTG = nx.from_pandas_edgelist(blueTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
    RTG = nx.from_pandas_edgelist(redTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    posB = nx.circular_layout(BTG)
    posR = nx.circular_layout(RTG)
    
    nx.draw(BTG, pos=posB, ax=axes[0], with_labels=True, node_size=2100, node_color="lightblue", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=blueTeamEdges, edge_color='gray')
    axes[0].set_title(blueTeamText)

    nx.draw(RTG, pos=posR, ax=axes[1], with_labels=True, node_size=2100, node_color="red", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=redTeamEdges, edge_color='gray')
    axes[1].set_title(redTeamText)

    # Match title
    plt.suptitle(titleText, fontsize=16)
    # plt.tight_layout()
    # plt.show()

    # save the figure
    output_path = f'../graphs/{matchRank}/{gameId}.png'
    plt.savefig(output_path, format='png', dpi=300)
    print(f"Image {output_path} saved successfully")
    plt.clf()

In [ ]:
print(valid_player_match_details_df.shape)
print(valid_player_match_details_df.columns)

In [ ]:
df = assistsMapDF[['gameId', 'teamId', 'participantId', 'intensity', 'totalTeamAllKills', 'totalTeamAllAssists', 'playerTotalIndegree', 'playerTotalOutdegree', 'teamIndegreeCentrality', 'teamOutdegreeCentrality', 'teamWeightCentralization', 'resistance']]
df.drop_duplicates(inplace=True)
print(df.shape)

valid_player_match_details_df = valid_player_match_details_df.merge(df, on=['gameId', 'teamId', 'participantId'], how='inner')
print(valid_player_match_details_df.shape)


### Export team level statistics to a CSV

In [ ]:
from sklearn.preprocessing import StandardScaler

# team averaged dataset
teamDF = valid_player_match_details_df[[
    'gameId', 'teamId', 'gameDuration', 'win', 'teamAverageRank', 'matchAverageRank', 
    'totalTeamKills', 'totalTeamEpicMonsterKills', 'totalTeamTurretKills', 'totalTeamAllAssists', 'totalTeamGold', 'totalTeamVisionScore', 'totalTeamChampExperience', 
    'totalTeamMinionsKilled', 'resistance', 'teamIndegreeCentrality', 'teamOutdegreeCentrality', 'teamWeightCentralization'
    ]]

teamDF.drop_duplicates(inplace=True) # redundant rows for 5 teammates

teamDF['goldPerMin'] = teamDF['totalTeamGold'] / teamDF['gameDuration']
teamDF['totalTeamMininionsPerMin'] = teamDF['totalTeamMinionsKilled'] / teamDF['gameDuration']
teamDF['totalTeamVisionPerMin'] = teamDF['totalTeamVisionScore'] / teamDF['gameDuration']
teamDF['totalTeamChampExpPerMin'] = teamDF['totalTeamChampExperience'] / teamDF['gameDuration']
teamDF['totalTeamAllAssistsPerMin'] = teamDF['totalTeamAllAssists'] / teamDF['gameDuration']

teamDFAllMetrics = teamDF.copy()
teamDFAllMetrics.to_csv('./teamAllMetrics.csv', index=False)

## Individual Statistics

In [ ]:
playerDF = valid_player_match_details_df[['gameId', 'teamId', 'participantId', 'teamPosition', 'win', 'gameDuration', 'champExperience', 'kills', 'assists', 'turretKills', 'epicMonsterKills', 'visionScore', 'goldEarned', 'totalMinionsKilled', 'playerTotalIndegree', 'playerTotalOUt']]
print(playerDF.shape)

# match max champ experience; gold earned
matchMaxChampExperience = playerDF.groupby(['gameId'])[['champExperience', 'goldEarned', 'totalMinionsKilled', 'visionScore']].max().reset_index()
matchMaxChampExperience.rename(columns={'champExperience': 'matchMaxChampExperience', 'goldEarned': 'matchMaxGoldEarned'}, inplace=True)
playerDF = playerDF.merge(matchMaxChampExperience, on='gameId', how='inner')

# match min champ experience; gold earned
matchMinChampExperience = playerDF.groupby(['gameId'])[['champExperience', 'goldEarned', 'totalMinionsKilled', 'visionScore']].min().reset_index()
matchMinChampExperience.rename(columns={'champExperience': 'matchMinChampExperience', 'goldEarned': 'matchMinGoldEarned'}, inplace=True)
playerDF = playerDF.merge(matchMinChampExperience, on=['gameId'], how='inner')


# min-max normaization
def expMinMaxNormalization(row):
    if ((row['champExperience'] == row['matchMinChampExperience']) & (row['champExperience'] == row['matchMaxChampExperience'])):
        return 0.5
    else:
        return ((row['champExperience'] - row['matchMinChampExperience']) / (row['matchMaxChampExperience'] - row['matchMinChampExperience']))
    
def goldMinMaxNormalization(row):
    if ((row['goldEarned'] == row['matchMinGoldEarned']) & (row['goldEarned'] == row['matchMaxGoldEarned'])):
        return 0.5
    else:
        return ((row['goldEarned'] - row['matchMinGoldEarned']) / (row['matchMaxGoldEarned'] - row['matchMinGoldEarned']))

# playerDF[['champExperience', 'goldEarned']] = scaler.fit_transform(playerDF[['champExperience', 'goldEarned']])

playerDF.head(20)